In [18]:
import json
import csv

In [2]:
def get_json(path : str):
    with open(path, "r") as f:
        return json.load(f)

In [17]:
def save_json(obj : any, path : str):
    with open(path, "w+") as f:
        json.dump(obj, f)

In [19]:
def save_csv(obj : list, path : str):
    headers = obj[0].keys() 
    with open(path, "w+") as f:
        writer = csv.DictWriter(f, headers)
        writer.writeheader()
        writer.writerows(obj)

In [3]:
legislators = get_json("data/recent_legislators_map.json")

In [22]:
senate_bills = get_json("data/senate_bills.json")

In [21]:
house_bills = get_json("data/house_bills.json")

In [29]:
def set_cosponsor_counts(bills : list):
    for bill in bills:
        sponsor = bill["sponsor"]
        if "thomas_id" in sponsor and sponsor["thomas_id"] in legislators:
            sponsor["party"] = legislators[sponsor["thomas_id"]]["party"]
        elif "bioguide_id" in sponsor and sponsor["bioguide_id"] in legislators:
            sponsor["party"] = legislators[sponsor["bioguide_id"]]["party"]
        else:
            sponsor["party"] = "unknown"
        cosponsor_party_count = {}
        for cosponsor in bill["cosponsors"]: 
            if "thomas_id" in cosponsor and cosponsor["thomas_id"] in legislators:
                cosponsor["party"] = legislators[cosponsor["thomas_id"]]["party"]
            elif "bioguide_id" in cosponsor and cosponsor["bioguide_id"] in legislators:
                cosponsor["party"] = legislators[cosponsor["bioguide_id"]]["party"]
            else:
                cosponsor["party"] = "unknown"
            if cosponsor["party"] not in cosponsor_party_count:
                cosponsor_party_count[cosponsor["party"]] = 0
            cosponsor_party_count[cosponsor["party"]] += 1
        bill["cosponsor_party_count"] = cosponsor_party_count

In [30]:
set_cosponsor_counts(senate_bills)
set_cosponsor_counts(house_bills)

In [34]:
parties = list(set([leg["party"] for leg in legislators.values()]))
parties

['Republican',
 'Independent',
 'Conservative',
 'Popular Democrat',
 'New Progressive',
 'multiple',
 'Democrat']

In [38]:
cosponsor_count_info = []
for bill in senate_bills + house_bills:
    count_info = {
        "bill_id" : bill["bill_id"],
        "type" :  bill["bill_type"],
        "sponsor_party" : bill["sponsor"]["party"],
        "congress" : bill["congress"]
    }
    for party in parties + ["unknown"]:
        count_info[party] = bill["cosponsor_party_count"][party] if party in bill["cosponsor_party_count"] else 0
    cosponsor_count_info.append(count_info)

In [41]:
save_csv(cosponsor_count_info, "data/cosponsor_party_count.csv")